In [100]:
import pandas as pd
import numpy as np

import tarfile

import time
import re

In [101]:
year = str(2018)
tar = tarfile.open(year + ".tar.gz")

In [114]:
keep_states = "WA US|OR US"#|ID US|MT US"#|CA US|NV US|WY US|UT US|AZ US"
ks_re = re.compile(keep_states)

min_temps = 365
n_signals = 365
knn = 8
dir_graph = False
file_name = 'temperatures' + year
# file_name = '../temperatures2003_3months'
file_name += '_knn' + str(knn)
if dir_graph:
    file_name += '_dir'

In [115]:
dfs = []
temp = []
windsp = []
prec = []
pres = []
attrs = ["TEMP", "WDSP", "PRCP", "STP"]

for member in tar.getmembers():
    f = tar.extractfile(member)
    df = pd.read_csv(f)
    f.close()
    df.dropna(inplace=True)

    if len(df) == 0:
        continue
    
    station_name = df.NAME.iloc[0]

    if ks_re.search(station_name) and len(df) >= n_signals and not (np.any(df[attrs].values == 999.9) or np.any(df[attrs].values == 99.99)):
        dfs.append(df)
        temp.append(df['TEMP'].values)
        windsp.append(df['WDSP'].values)
        prec.append(df['PRCP'].values)
        pres.append(df['STP'].values)

assert len(dfs) > 0, "No data with specified criteria"
df = pd.concat(dfs)
temps = np.array(temp)
windsps = np.array(windsp)
precs = np.array(prec)
press = np.array(pres)

In [116]:
temps.shape, windsps.shape, precs.shape, press.shape

((28, 365), (28, 365), (28, 365), (28, 365))

In [105]:
# # Constants
# Limit temperature in Fº
MAX_TEMP = 140
MIN_TEMP = -30
assert np.all(temps <= MAX_TEMP) and np.all(temps >= MIN_TEMP)

In [106]:
stations = df.drop_duplicates('NAME')
N = len(stations)

In [107]:
# Read stations coordinates and convert to radians
Coords = np.zeros((N, 2))
Coords[:, 0] = stations.LONGITUDE.to_numpy()*np.pi/180
Coords[:, 1] = stations.LATITUDE.to_numpy()*np.pi/180

# Earth radius in km
R_EARTH = 6371
# Coordinates in km
Coords_km = np.zeros((N, 2))
Coords_km[:, 0] = R_EARTH*Coords[:, 0]*np.cos(Coords[:, 1])
Coords_km[:, 1] = R_EARTH*Coords[:, 1]

In [108]:
# For geodesic distance in km
D = np.zeros((N, N))
for i in range(N):
    for j in range(i+1, N):
        D[i, j] = np.linalg.norm(Coords_km[i, :] - Coords_km[j, :])
D = D + D.T

In [109]:
P = np.exp(-D/np.sum(D)*N**2)
P_n = np.sum(P, axis=0)
np.fill_diagonal(D, np.inf)

idx = D.argsort()[:, :knn]
A = np.zeros(D.shape)
for i in range(N):
    A[i, idx[i, :]] = P[i, idx[i, :]]/P_n[idx[i, :]]
    if not dir_graph:
        A[idx[i, :], i] = A[i, idx[i, :]]

In [110]:
A_bin = np.zeros(A.shape)
A_bin[A != 0] = 1
print('Zeros:', np.sum(A == 0))
print('Non Zeros:', np.sum(A != 0))
print('Mean degree of A:', np.mean(np.sum(A_bin, axis=0)))

Zeros: 2396
Non Zeros: 520
Mean degree of A: 9.62962962962963


In [111]:
file_name += '_N' + str(N)
np.savez(file_name, A=A, temps=temps, precs=precs, windsps=windsps, press=press, Coords=Coords,
         Coords_km=Coords_km, A_bin=A_bin, D=D)
print('File saved as ', file_name)

File saved as  temperatures2018_knn8_N54
